In [ ]:
!pip install requests beautifulsoup4 lxml pandas tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Базовый URL для рубрики "Технологии"
base_url = "https://lifehacker.ru/topics/technology/?page="

# Список для хранения данных
articles = []

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Получение данных с 10 страниц
for page_num in range(1, 10):
    url = f"{base_url}{page_num}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Ошибка при запросе страницы {page_num}. Код ответа: {response.status_code}")
        continue

    soup = BeautifulSoup(response.text, 'lxml')

    # Ищем ссылки на статьи
    article_links = [a['href'] for a in soup.find_all('a', class_='lh-small-article-card__link')]

    # преобразуем ссылки и получаем данные статей
    for link in tqdm(article_links, desc=f"Парсинг страницы {page_num}"):
        # Обработка относительных ссылок
        full_link = f"https://lifehacker.ru{link}" if link.startswith('/') else link

        # Переход по ссылке и парсинг содержимого статьи
        article_response = requests.get(full_link, headers=headers)
        if article_response.status_code != 200:
            print(f"Ошибка при запросе статьи {full_link}. Код ответа: {article_response.status_code}")
            continue

        article_soup = BeautifulSoup(article_response.text, 'lxml')

        # ИЗВЛЕЧЕНИЕ СОДЕРЖИМОГО
        # Заголовок
        title_tag = article_soup.find('h1', class_='article-card__title')
        title = title_tag.text.strip() if title_tag else "Заголовок не найден"

        # Основной текст
        content_div = article_soup.find(class_='single-article__post-content single-article__content-container')
        text = content_div.text.strip() if content_div else "Текст статьи не найден"

        # Сохранение данных
        articles.append({"title": title, "text": text, "link": full_link})

# Создание DataFrame
df = pd.DataFrame(articles)
df.to_csv('lifehacker_technology_articles.csv', index=False)
print("\nДанные успешно собраны и сохранены в файл lifehacker_technology_articles.csv")


Парсинг страницы 1: 100%|██████████| 30/30 [00:27<00:00,  1.09it/s]


Данные успешно собраны и сохранены в файл lifehacker_technology_articles.csv
